In [ ]:
import cv2
import numpy as np
import glob
import pickle
from sklearn.externals import joblib
from keras.applications.vgg16 import preprocess_input
from sklearn import preprocessing
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import glob
import keras
from keras.models       import Model
from keras.applications.densenet import DenseNet121
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet121
from keras.models import Model
import glob



In [ ]:
imgs=[]
final_output_84_84=[]
lables=[]
for filefilepath in glob.iglob('final_casia/*'):
    
    
    if filefilepath[-1] == 'g':
        
        img	= cv2.imread(filefilepath)
        imgs_colored=cv2.imread(filefilepath)
        #img=cv2.resize(img,(200,150))
        #imgs_colored.append(img)

        print(filefilepath)
        #print(filefilepath[19:-6])
        #print(filefilepath[-5])
        split = filefilepath.split(".")
        #print(split)
        print(split[0][12:])
        print(split[1])

        label=split[0][12:]
        example_number = split[1]
        imgs.append([imgs_colored,int(label)])
        #final_output_84_84.append(imgs_colored)
        #lables.append(int(label))
    
import random

random.shuffle(imgs)

for i,j in imgs:
        final_output_84_84.append(i)
        lables.append(j)
    
print(len(final_output_84_84))
print(len(lables))



In [ ]:
#lables

In [ ]:
img_size=(70,70)

In [ ]:
y=keras.utils.to_categorical(lables, num_classes=1000, dtype='float32')


In [ ]:
final_output_84_84 = np.array(final_output_84_84,dtype="float16")/255


In [ ]:
from keras.models       import Model
from keras.applications.densenet import DenseNet201
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception

#original_model = VGG16(include_top=False)
original_model = DenseNet201(include_top=False)
#original_model = InceptionV3(include_top=False)
#original_model = InceptionResNetV2(include_top=False)
#original_model = Xception(include_top=False)

bottleneck_input  = original_model.get_layer(index=0).input
bottleneck_output = original_model.get_layer(index=-59).output
bottleneck_model  = Model(inputs=bottleneck_input,  outputs=bottleneck_output)

bottleneck_model.summary()


In [ ]:

#bottleneck_model = InceptionV3(include_top=False)
#Inception.summary()

In [ ]:
#bottleneck_model    = VGG16(include_top=False)
#bottleneck_model.summary()
'''
bottleneck_model   = DenseNet201(include_top=False)
bottleneck_model.summary()
'''

In [ ]:
test = final_output_84_84[0]
test = test.reshape(1, img_size[0], img_size[1],3)
test_shape = bottleneck_model.predict(test).shape
print(test_shape)
shape = (final_output_84_84.shape[0],test_shape[1],test_shape[2],test_shape[3])
print(shape)
print(shape[1:])

In [ ]:
bottelneck_features = []
for i in final_output_84_84:
    
    i = i.reshape(1, img_size[0], img_size[1],3)
    bottelneck_features.append(bottleneck_model.predict(i))
    print(len(bottelneck_features))
    
    #if len(bottelneck_features)==20000:
        #break
    
bottelneck_features=np.array(bottelneck_features)
print(bottelneck_features.shape)

In [ ]:
#print(np.amax(ubiris_features)  )
bottelneck_features =  bottelneck_features.reshape(shape) #4, 6, 512
print(bottelneck_features.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm

(X_train, X_test, y_train, y_test) = train_test_split(final_output_84_84, lables,test_size=0.05, random_state=1250)

In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
from keras.layers import Dense, Flatten,Dropout
from keras.models import Sequential
from keras import optimizers


model = Sequential()
model.add(Flatten(input_shape=(final_output_84_84.shape[1:])))#4, 6, 512
#model.add(Dense(500, activation='relu'))
#model.add(Dropout(0.8))
model.add(Dense(1000, activation='softmax'))
sgd = optimizers.SGD(lr=0.1)
#adam = optimizers.Adam(learning_rate=0.001)


model.compile(loss='categorical_crossentropy', optimizer='adam', 
                  metrics=['accuracy'])
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='casia_dense_net_0.2_adam.hdf5', verbose=1, save_best_only=True)
#X_train, X_test, y_train, y_test

history=model.fit(X_train,y_train, batch_size=32, epochs=30, shuffle=True,validation_data=(X_test, y_test), verbose=1 ,callbacks=[checkpointer])

In [ ]:
import matplotlib.pyplot as plt
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve
y_pred_keras = keras_model.predict(X_test).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred_keras)

In [ ]:
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()